In [ ]:
import pandas as pd
import numpy as np
import os
import keras

In [ ]:
combined = pd.read_csv("part_1_cleanusingcountencoding.csv", index_col=0)
combined.head()

,Nationality,Age,DaysSinceCreation,AverageLeadTime,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,...,Age_exponential,DistributionChannel_Direct,DistributionChannel_Electronic Distribution,DistributionChannel_Travel Agent/Operator,MarketSegment_Complementary,MarketSegment_Corporate,MarketSegment_Direct,MarketSegment_Groups,MarketSegment_Other,MarketSegment_Travel Agent/Operator
0,7612,51.0,150,45,1,0,3,8,5,151,...,26.483486,0,0,0,0,1,0,0,0,0
2,10034,31.0,1095,0,0,0,0,0,0,-1,...,17.490467,0,0,1,0,0,0,0,0,1
3,12293,60.0,1095,93,0,0,1,10,5,1100,...,30.324434,0,0,1,0,0,0,0,0,1
4,12293,51.0,1095,0,0,0,0,0,0,-1,...,26.483486,0,0,1,0,0,0,0,0,1
5,253,54.0,1095,58,0,0,1,4,2,1097,...,27.775473,0,0,1,0,0,0,0,1,0


In [ ]:
combined.shape

(78473, 38)

In [ ]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78473 entries, 0 to 82579
Data columns (total 38 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Nationality                                  78473 non-null  int64  
 1   Age                                          78473 non-null  float64
 2   DaysSinceCreation                            78473 non-null  int64  
 3   AverageLeadTime                              78473 non-null  int64  
 4   BookingsCanceled                             78473 non-null  int64  
 5   BookingsNoShowed                             78473 non-null  int64  
 6   BookingsCheckedIn                            78473 non-null  int64  
 7   PersonsNights                                78473 non-null  int64  
 8   RoomNights                                   78473 non-null  int64  
 9   DaysSinceLastStay                            78473 non-null  int64  
 10

In [ ]:
combined.drop(['Age_log','Age_reciprocal', 'Age_sqaure', 'Age_exponential'],axis=1,inplace=True)

In [ ]:
combined.isnull().sum()

Nationality                                    0
Age                                            0
DaysSinceCreation                              0
AverageLeadTime                                0
BookingsCanceled                               0
BookingsNoShowed                               0
BookingsCheckedIn                              0
PersonsNights                                  0
RoomNights                                     0
DaysSinceLastStay                              0
DaysSinceFirstStay                             0
SRHighFloor                                    0
SRLowFloor                                     0
SRAccessibleRoom                               0
SRMediumFloor                                  0
SRBathtub                                      0
SRShower                                       0
SRCrib                                         0
SRKingSizeBed                                  0
SRTwinBed                                      0
SRNearElevator      

In [ ]:
combined[combined.duplicated()].shape

(1655, 34)

In [ ]:
# removing duplicate rows
combined = combined.drop_duplicates()
combined[combined.duplicated()].shape

(0, 34)

In [ ]:
combined.nunique().to_frame(name = 'Number of unique values')

,Number of unique values
Nationality,86
Age,93
DaysSinceCreation,1083
AverageLeadTime,402
BookingsCanceled,6
BookingsNoShowed,4
BookingsCheckedIn,27
PersonsNights,53
RoomNights,44
DaysSinceLastStay,1100


In [ ]:
# removing outliers
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(combined)
y_noano = clf.predict(combined)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

combined = combined.iloc[y_noano[y_noano['Top'] == 1].index.values]
combined.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", combined.shape[0])

Number of Outliers: 16660
Number of rows without outliers: 60158


In [ ]:
y = combined['BookingsCheckedIn']
combined.drop(['BookingsCheckedIn'], axis=1, inplace=True)
X = combined

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Select categorical columns with relatively low cardinality 
categorical_cols = [cname for cname in X.columns if
                    X[cname].nunique() < 100 and 
                    X[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X.columns if 
                X[cname].dtype in ['int64', 'float64']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
y_train.values.reshape(1,-1)

array([[1, 1, 1, ..., 1, 1, 0]])

In [ ]:
# standarisation: We use the MinMaxScaler() from sklearn library
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

# fit and transform on train data
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
# transform test data
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test,y_test)

0.7321735755059857

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, reg.predict(X_test))

0.05321602815460586

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)
regressor.score(X_test,y_test)

0.9406034134514768

In [ ]:
mean_squared_error(y_test, regressor.predict(X_test))

0.01180186170212766

In [ ]:
## define a gradient boosting model
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
xgb = XGBRegressor(model__colsample_bytree= 0.7, model__learning_rate= 0.01, model__max_depth= 5, model__n_estimators= 4000, model__reg_lambda= 0.9, model__subsample= 0.4)
xgb.fit(X_train,y_train)
xgb.score(X_test,y_test)

[07:45:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9620419413224748

In [ ]:
mean_squared_error(y_test, xgb.predict(X_test))

0.007542112855718272

## Neural Network Regression Analysis :

In [ ]:
import math
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [ ]:
# another model
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

seed = 7
np.random.seed(seed)

# Model
model1 = Sequential()
model1.add(Dense(200, input_dim=33, kernel_initializer='normal', activation='relu'))
model1.add(Dense(100, kernel_initializer='normal', activation='relu'))
model1.add(Dense(50, kernel_initializer='normal', activation='relu'))
model1.add(Dense(25, kernel_initializer='normal', activation='relu'))
model1.add(Dense(1, kernel_initializer='normal'))
# Compile model
model1.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01))

feature_cols = X_train
labels = y_train.values

model1.fit(np.array(feature_cols), np.array(labels), epochs=30, batch_size=10)

Epoch 1/30
5415/5415 [==============================] - 16s 3ms/step - loss: 0.0167
Epoch 2/30
5415/5415 [==============================] - 12s 2ms/step - loss: 0.0142
Epoch 3/30
5415/5415 [==============================] - 11s 2ms/step - loss: 0.0130
Epoch 4/30
5415/5415 [==============================] - 12s 2ms/step - loss: 0.0130
Epoch 5/30
5415/5415 [==============================] - 12s 2ms/step - loss: 0.0129
Epoch 6/30
5415/5415 [==============================] - 12s 2ms/step - loss: 0.0128
Epoch 7/30
5415/5415 [==============================] - 12s 2ms/step - loss: 0.0128
Epoch 8/30
5415/5415 [==============================] - 12s 2ms/step - loss: 0.0127
Epoch 9/30
5415/5415 [==============================] - 17s 3ms/step - loss: 0.0128
Epoch 10/30
5415/5415 [==============================] - 12s 2ms/step - loss: 0.0127
Epoch 11/30
5415/5415 [==============================] - 12s 2ms/step - loss: 0.0126
Epoch 12/30
5415/5415 [==============================] - 12s 2ms/step - lo

In [ ]:
# Evaluation on the test set created by train_test_split
model1.evaluate(np.array(X_test), np.array(y_test))

188/188 [==============================] - 0s 1ms/step - loss: 0.0093


0.009316489100456238

In [ ]:
### creating pickle file for model1
import pickle
pickle_out = open("modelDeepLearning.pkl","wb")
pickle.dump(model1, pickle_out)
pickle_out.close()